In [1]:
import sys
sys.path.extend(["/home/matt/code/logit/"])
import reflex as rx
from sqlmodel import create_engine, Session, select
from weightsapp import LoggedExercise
import datetime
from datetime import datetime

In [2]:
sqlite_fn = "/home/matt/code/logit/data.db"
sqlite_url = f"sqlite:///{sqlite_fn}"

engine = create_engine(sqlite_url, echo=True)


In [3]:
with Session(engine) as session:
    statement = select(LoggedExercise)
    results = session.exec(statement)
    for r in results:
        print(r)

2023-09-22 11:24:01,691 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-22 11:24:01,693 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id, loggedexercise.created_datetime, loggedexercise.ename, loggedexercise.enum, loggedexercise.reps, loggedexercise.weight 
FROM loggedexercise
2023-09-22 11:24:01,693 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ()
ename='benchpress' reps=10 id=1 enum=0 created_datetime='20-09-23' weight=40.0
ename='benchpress' reps=7 id=2 enum=1 created_datetime='20-09-23' weight=50.0
ename='benchpress' reps=5 id=3 enum=2 created_datetime='20-09-23' weight=60.0
ename='deadlift' reps=5 id=4 enum=0 created_datetime='20-09-23' weight=70.0
ename='deadlift' reps=5 id=5 enum=1 created_datetime='20-09-23' weight=90.0
ename='deadlift' reps=5 id=6 enum=2 created_datetime='20-09-23' weight=130.0
ename='benchpress' reps=10 id=7 enum=0 created_datetime='22-09-23' weight=20.0
ename='benchpress' reps=10 id=8 enum=1 created_datetime='22-09-23' weight=40.0
ename=

In [4]:
import pandas as pd
def _log_as_dataframe() -> pd.DataFrame:
    
    with Session(engine) as session:
        ex_list = (
            session.query(LoggedExercise).all()
        )
    ex_list = [ex._li() for ex in ex_list]    
    
    data_columns = ["date", "ename", "enum", "reps", "kg"]
    data_types = {'ename':str, 'enum': int, 'reps': int, 'kg': float}
    return pd.DataFrame(ex_list,
                            columns=data_columns).astype(data_types)

In [5]:
# dummy_data = [
#     ["20-09-23", "benchpress", "0", "10", "40"],
#     ["20-09-23", "benchpress", "1", "7", "50"],
#     ["20-09-23", "benchpress", "2", "5", "60"],

#     ["20-09-23", "deadlift", "0", "5", "70"],
#     ["20-09-23", "deadlift", "1", "5", "90"],
#     ["20-09-23", "deadlift", "2", "5", "130"],

#     ["22-09-23", "benchpress", "0", "10", "20"],
#     ["22-09-23", "benchpress", "1", "10", "40"],
#     ["22-09-23", "benchpress", "2", "6", "60"],
#     ["22-09-23", "benchpress", "3", "6", "60"],
#     ["22-09-23", "benchpress", "4", "6", "60"],
    
#     ["23-09-23", "deadlift", "0", "5", "80"],
#     ["23-09-23", "deadlift", "1", "5", "100"],
#     ["23-09-23", "deadlift", "2", "5", "130"],
#     ["23-09-23", "deadlift", "3", "5", "130"],
# ]

In [6]:
# for (date,ename,enum,rep,kg) in dummy_data:



#     with Session(engine) as session:
        

#         session.add(
#             LoggedExercise(
#             # self.total_set_number,
#             # date.today().strftime("%d-%m-%y"),
#                 created_datetime=date,
#                 ename=ename,
#                 enum=enum,#self.exercise_counter[self.current_exercise[selector_id]],
#                 reps=rep,
#                 weight=kg#
#             )
#         )
#         session.commit()

In [7]:
df = _log_as_dataframe()

# df['date'] = df.date.apply(lambda x: datetime.strptime(x, "%d-%m-%y"))

2023-09-22 11:24:02,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-22 11:24:02,097 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id AS loggedexercise_id, loggedexercise.created_datetime AS loggedexercise_created_datetime, loggedexercise.ename AS loggedexercise_ename, loggedexercise.enum AS loggedexercise_enum, loggedexercise.reps AS loggedexercise_reps, loggedexercise.weight AS loggedexercise_weight 
FROM loggedexercise
2023-09-22 11:24:02,097 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()
2023-09-22 11:24:02,099 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
from data_analysis import load_intensity_figure, progression_figure

ename='deadlift'
fg = load_intensity_figure(ename, df)

In [9]:
fg2 = progression_figure(ename, df, df)

In [22]:
fg

In [19]:
fg2

In [12]:
# import plotly.express as px
# import plotly.graph_objects as go

# fig = go.Figure(
#     data=[go.Line(x=df['date'], y=df['reps'])],
#         layout=go.Layout(
#         title=go.layout.Title(text="A Figure Specified By A Graph Object")
#     )
# )
# fig.show()

In [13]:
import numpy as np
import plotly.graph_objects as go

df['load'] = df.reps * df.kg

grpby = df.groupby(['date','ename'], as_index=False).aggregate(
    vol=("reps", 'sum'),
    intensity=("kg", 'max'),
    load=("load",'sum')
)


In [14]:
grpby

,date,ename,vol,intensity,load
0,2023-09-20,benchpress,22,60.0,1050.0
1,2023-09-20,deadlift,15,130.0,1450.0
2,2023-09-21,Scan,5,70.0,350.0
3,2023-09-21,benchpress,5,70.0,350.0
4,2023-09-21,kb swing,10,28.0,280.0
5,2023-09-22,benchpress,38,60.0,1680.0
6,2023-09-23,deadlift,20,130.0,2200.0


In [15]:
grpby[grpby.ename=='benchpress']

,date,ename,vol,intensity,load
0,2023-09-20,benchpress,22,60.0,1050.0
3,2023-09-21,benchpress,5,70.0,350.0
5,2023-09-22,benchpress,38,60.0,1680.0


In [16]:
ex = 'benchpress'
metrics = ['load', 'intensity']#, 'vol']

lines = [
    go.Line(name=metric, x=grpby[grpby.ename==ex]['date'], y=grpby[metric])
    for metric in metrics
]

fig = go.Figure(data = lines)

fig.show()

print(grpby)

/home/matt/venvs/py39/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/matt/venvs/py39/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



        date       ename  vol  intensity    load
0 2023-09-20  benchpress   22       60.0  1050.0
1 2023-09-20    deadlift   15      130.0  1450.0
2 2023-09-21        Scan    5       70.0   350.0
3 2023-09-21  benchpress    5       70.0   350.0
4 2023-09-21    kb swing   10       28.0   280.0
5 2023-09-22  benchpress   38       60.0  1680.0
6 2023-09-23    deadlift   20      130.0  2200.0


In [17]:
from plotly.subplots import make_subplots

ex = 'benchpress'
metrics = ['load', 'intensity']#, 'vol']


fig = make_subplots(specs=[[{"secondary_y":True}]])

data = grpby[grpby.ename==ex]

y_label = ['Total Kg Moved', 'Max Kg Moved']

# Add traces
for i,metric in enumerate(metrics):
    fig.add_trace(
        go.Line(
            x=data['date'], y=data[metric], name=metric
        ),
        secondary_y=i==1
    )
    
    fig.update_yaxes(title_text=y_label[i], secondary_y=(i==1))

fig.update_layout(title_text=ex)
fig.update_xaxes(title_text='Date')



/home/matt/venvs/py39/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/matt/venvs/py39/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [18]:
fig.to_dict()['layout']

{'template': {'data': {'histogram2dcontour': [{'type': 'histogram2dcontour',
     'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'colorscale': [[0.0, '#0d0887'],
      [0.1111111111111111, '#46039f'],
      [0.2222222222222222, '#7201a8'],
      [0.3333333333333333, '#9c179e'],
      [0.4444444444444444, '#bd3786'],
      [0.5555555555555556, '#d8576b'],
      [0.6666666666666666, '#ed7953'],
      [0.7777777777777778, '#fb9f3a'],
      [0.8888888888888888, '#fdca26'],
      [1.0, '#f0f921']]}],
   'choropleth': [{'type': 'choropleth',
     'colorbar': {'outlinewidth': 0, 'ticks': ''}}],
   'histogram2d': [{'type': 'histogram2d',
     'colorbar': {'outlinewidth': 0, 'ticks': ''},
     'colorscale': [[0.0, '#0d0887'],
      [0.1111111111111111, '#46039f'],
      [0.2222222222222222, '#7201a8'],
      [0.3333333333333333, '#9c179e'],
      [0.4444444444444444, '#bd3786'],
      [0.5555555555555556, '#d8576b'],
      [0.6666666666666666, '#ed7953'],
      [0.7777777777777778, '#fb9f3a